# Compute the atom centered Coulomb Matrix representation

To install QML library and compare rascal results with it
+ pip install qml ase

To install rascal:
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release ..
+ make install -j 4

In [1]:
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
import qml
from ase.io import read
from ase.visualize import view
import numpy as np
import sys, os
from ase.build import make_supercell

In [5]:
import sys, os
sys.path.insert(0,'../build_bindings/')
import rascal as rc

In [6]:
from rascal.representation import SortedCoulombMatrix

In [7]:
from rascal.representation.base import _representations

In [8]:
_representations

{'sortedcoulomb_strict_neighbourlist_centers': rascal.lib._rascal.RepresentationManager.SortedCoulomb_Strict_NeighbourList_Centers,
 'sphericalexpansion_strict_neighbourlist_centers': rascal.lib._rascal.RepresentationManager.SphericalExpansion_Strict_NeighbourList_Centers}

In [ ]:
# load a small subset of structures from QM9
frames = read('../tests/reference_data/dft-smiles_500.xyz',':')
cutoff = 3.
rep = SortedCoulombMatrix(cutoff,sort='rownorm')

In [ ]:
new_frames = []
aa = []
for frame in frames:
    new_frames.append(make_supercell(frame,np.eye(3)*1))
print(np.max(list(map(len,new_frames))))

In [ ]:
# have a look at them
view(frames)

In [ ]:
# Compute the sorted coulomb matrices for the list of structures
%timeit -n 3 -r 3 features = rep.transform(new_frames)

In [ ]:
# extract the feature matrix to compare with 
features = rep.transform(new_frames)
test = features.get_feature_matrix().T

In [ ]:
features.shape

In [ ]:
# To get the coulomb matrices from QML
def get_coulomb_ref(frame,size,cutoff,flavour):
    from qml.representations import generate_atomic_coulomb_matrix
    
    nuclear_charges = frame.get_atomic_numbers()
    coordinates = frame.get_positions()
    
    
    cm = generate_atomic_coulomb_matrix(nuclear_charges, coordinates, size = size, sorting = flavour,
    central_cutoff = cutoff, central_decay = -1, interaction_cutoff = 1e6, interaction_decay = -1,indices = None)
    
    return cm
def get_coulomb_refs(frames,size,cutoff,flavour='distance'):  
    cms = []
    for frame in frames:
        cms.append(get_coulomb_ref(frame,size,cutoff,flavour))
    return np.vstack(cms)

In [ ]:
# Compare with reference
%timeit -n 3 -r 3 ref = get_coulomb_refs(new_frames,rep.size,cutoff,flavour='row-norm')
ref = get_coulomb_refs(new_frames,rep.size,cutoff,flavour='row-norm')
np.allclose(test,ref)

In [ ]:
from rascal.utils import fps

In [ ]:
sparse_indices,sparse_minmax_d2 = fps(test,Nselect=500,starting_index=None,flavour='simple')

In [ ]:
plt.plot(sparse_minmax_d2)